# Day 7: Model Quantization and Running LLMs Locally

This notebook teaches you how to run large language models efficiently on resource-constrained devices using quantization techniques. You'll learn how to:

- Install and configure quantization libraries (bitsandbytes, accelerate)
- Load models with 4-bit quantization to reduce memory requirements
- Compare different open-source models (Llama, Phi, Qwen, DeepSeek)
- Implement streaming generation for real-time text output
- Optimize inference through quantization, tokenization, and memory management

The practical exercise involves loading different model architectures with quantization, generating responses with streaming, and comparing the memory footprint and performance of various small-to-medium sized open-source LLMs.


<a href="https://colab.research.google.com/github/Istiaq-Fuad/learning-llm-engineering/blob/main/day7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [15]:
!pip install -q --upgrade bitsandbytes accelerate

In [ ]:
from google.colab import userdata
from huggingface_hub import login
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TextStreamer,
    BitsAndBytesConfig,
)
import torch
import gc

## Setup imports & authentication


In [ ]:
hf_token = userdata.get("HF_TOKEN")
login(hf_token, add_to_git_credential=True)

In [18]:
LLAMA = "meta-llama/Llama-3.2-1B-Instruct"

PHI = "microsoft/Phi-4-mini-instruct"
QWEN = "Qwen/Qwen3-4B-Instruct-2507"
DEEPSEEK = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

In [ ]:
messages = [{"role": "user", "content": "Tell a joke for a room of Data Scientists"}]

In [ ]:
# Quantization Config - this allows us to load the model into memory and use less memory

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
)

## Run Llama with quantization


In [21]:
# Tokenizer

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [22]:
inputs

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1313,   3799,    220,   2366,     20,    271, 128009, 128006,
            882, 128007,    271,  41551,    264,  22380,    369,    264,   3130,
            315,   2956,  57116, 128009]], device='cuda:0')

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    LLAMA, device_map="auto", quantization_config=quant_config
)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [24]:
memory = model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:,.1f} MB")

Memory footprint: 1,012.0 MB


In [25]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm

In [26]:
outputs = model.generate(inputs, max_new_tokens=80)
outputs[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


tensor([128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
            25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
           220,   1313,   3799,    220,   2366,     20,    271, 128009, 128006,
           882, 128007,    271,  41551,    264,  22380,    369,    264,   3130,
           315,   2956,  57116, 128009, 128006,  78191, 128007,    271,   8586,
           596,    264,  22380,  41891,    311,    264,   3130,   2539,    315,
           828,  14248,   1473,  10445,   1550,    279,    828,  28568,   1464,
           709,    449,    813,  23601,   1980,  18433,   1364,    574,   2744,
          4560,    311,  30536,    872,   5133,     11,    719,    304,    279,
           842,     11,    433,    574,   1120,    264,    828,  32505,   5597,
           311,    733,   1077,   8821,   5627,     13, 128009],
       device='cuda:0')

In [27]:
tokenizer.decode(outputs[0])

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 22 Dec 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nTell a joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHere's a joke tailored to a room full of data scientists:\n\nWhy did the data scientist break up with his girlfriend?\n\nBecause she was always trying to optimize their relationship, but in the end, it was just a data-driven decision to go her separate ways.<|eot_id|>"

In [28]:
del model, inputs, tokenizer, outputs
gc.collect()
torch.cuda.empty_cache()

## Reusable generation function with multiple models


In [ ]:
# Wrapping everything in a function - and adding Streaming and generation prompts


def generate(model, messages, quant=True, max_new_tokens=80):
    tokenizer = AutoTokenizer.from_pretrained(model)
    tokenizer.pad_token = tokenizer.eos_token
    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to("cuda")
    attention_mask = torch.ones_like(input_ids, dtype=torch.long, device="cuda")
    streamer = TextStreamer(tokenizer)
    if quant:
        model = AutoModelForCausalLM.from_pretrained(
            model, quantization_config=quant_config
        ).to("cuda")
    else:
        model = AutoModelForCausalLM.from_pretrained(model).to("cuda")
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        streamer=streamer,
    )

In [30]:
generate(PHI, messages)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

<|user|>Tell a joke for a room of Data Scientists<|end|><|assistant|>Sure, here's a joke tailored for data scientists:

Why did the data scientist break up with the computer?

Because it kept dumping data on them, and they just couldn't handle the relationship anymore!<|end|>


In [32]:
generate(QWEN, messages)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

<|im_start|>user
Tell a light-hearted joke for a room of Data Scientists<|im_end|>
<|im_start|>assistant
Sure! Here's a light-hearted, data-savvy joke just for the data scientists in the room:

---

Why did the data scientist break up with their dataset?

Because it was *correlated* with a lot of missing values and just couldn’t commit to a *clean* relationship!

😄

*(Bonus points if you chuckle at the pun on "correlated" and "committed.")


In [33]:
generate(DEEPSEEK, messages, quant=False, max_new_tokens=500)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<｜begin▁of▁sentence｜><｜User｜>Tell a light-hearted joke for a room of Data Scientists<｜Assistant｜><think>
Okay, so I need to come up with a light-hearted joke for a room of data scientists. Hmm, data scientists are all about numbers, graphs, and maybe some humor. Let me think about what kind of light-hearted jokes they might find funny.

First, maybe something related to data analysis. Like, how they deal with data, how they crunch numbers, and maybe some funny scenarios where they face problems. But it needs to be light-hearted, so probably not too serious or too serious.

Let me brainstorm some scenarios they might encounter. Maybe they're working on a dataset, or they're trying to solve a problem. For example, if they're analyzing customer data, maybe they're trying to find trends or predict something. Or maybe they're dealing with a messy dataset, which could lead to funny situations.

I could also think about the tools they use, like Python, R, SQL. Maybe something about how they u